In [2]:
'''
ECSE 556 Homework 1
Clustering
Gian Favero
October 6th, 2023
'''

# Common imports
import numpy as np
import matplotlib.pyplot as plt
import time

# import data
data = np.genfromtxt('Data/gdsc_expr_postCB.csv', delimiter=',')
data = data[1:,1:]
data = np.transpose(data)

# Clustering

Agglomerative Clustering

In [3]:
from sklearn.cluster import AgglomerativeClustering

# Construct the clustering object
agglomerative_clustering = AgglomerativeClustering(n_clusters=3)

# Fit the data
agglomerative_clustering.fit(data)

# Get the labels
agglo_labels = agglomerative_clustering.labels_

K-Means Clustering

In [12]:
from sklearn.cluster import KMeans

# Construct the clustering object
k_means = KMeans(n_clusters=3)

# Fit the data
k_means.fit(data)

# Get the labels and cluster centers
k_cluster_centers = k_means.cluster_centers_
k_labels = k_means.labels_

c:\Users\gmari\Documents\Repos\ECSE556\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Similarities

In [13]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import rand_score
from sklearn.metrics import adjusted_rand_score
import prettytable as pt

# Jaccard score for each pair of clusters
jaccards = []
for i in range(3):
    for j in range(3):
        jaccards.append(jaccard_score(agglo_labels == i, k_labels == j).round(3))

# Make table of jaccard scores
default_t = pt.PrettyTable()
default_t.field_names = ['', 'Cluster 1', 'Cluster 2', 'Cluster 3']
default_t.title = 'Jaccard Score for Each Pair of Clusters (Default)'

for i in range(3):
    default_t.add_row([f'Cluster {i+1}', jaccards[i], jaccards[i+3], jaccards[i+6]])

print(default_t)

# Rand score for agglomerative and k-means clustering
def_rand = rand_score(agglo_labels, k_labels).round(3)
print("Rand Score: ", def_rand)

# Adjusted rand score for agglomerative and k-means clustering
def_adj_rand = adjusted_rand_score(agglo_labels, k_labels).__round__(3)
print("Adjusted Rand Score: ", def_adj_rand)


+---------------------------------------------------+
| Jaccard Score for Each Pair of Clusters (Default) |
+------------+------------+------------+------------+
|            | Cluster 1  | Cluster 2  | Cluster 3  |
+------------+------------+------------+------------+
| Cluster 1  |   0.019    |    0.0     |   0.855    |
| Cluster 2  |   0.887    |    0.0     |    0.05    |
| Cluster 3  |    0.0     |    1.0     |    0.0     |
+------------+------------+------------+------------+
Rand Score:  0.913
Adjusted Rand Score:  0.815


#### Analysis of Agglomerative Clustering

Euclidean Affinity for Agglomerative Clustering

In [14]:
# Eucledian distance
euc_agglo = AgglomerativeClustering(n_clusters=3, linkage='average', metric='euclidean')
euc_agglo.fit(data)
euc_labels = euc_agglo.labels_

# Jaccard score for each pair of clusters
euc_jaccards = []
for i in range(3):
    for j in range(3):
        euc_jaccards.append(jaccard_score(euc_labels == i, k_labels == j).round(3))

# Make table of jaccard scores
euc_t = pt.PrettyTable()
euc_t.field_names = ['', 'Cluster 1', 'Cluster 2', 'Cluster 3']
euc_t.title = 'Jaccard Score for Each Pair of Clusters (Agglomerative + Euclidean)'

for i in range(3):
    euc_t.add_row([f'Cluster {i+1}', euc_jaccards[i], euc_jaccards[i+3], euc_jaccards[i+6]])

print(euc_t)

# Rand score
euc_rand = rand_score(euc_labels, k_labels).round(3)
print("Rand Score: ", euc_rand)

# Adjusted rand score
euc_adj_rand = adjusted_rand_score(euc_labels, k_labels).__round__(3)
print("Adjusted Rand Score: ", euc_adj_rand)

+---------------------------------------------------------------------+
| Jaccard Score for Each Pair of Clusters (Agglomerative + Euclidean) |
+----------------+----------------+----------------+------------------+
|                |   Cluster 1    |   Cluster 2    |    Cluster 3     |
+----------------+----------------+----------------+------------------+
|   Cluster 1    |     0.342      |      0.0       |      0.018       |
|   Cluster 2    |     0.482      |     0.002      |       0.0        |
|   Cluster 3    |     0.174      |      0.0       |       0.0        |
+----------------+----------------+----------------+------------------+
Rand Score:  0.384
Adjusted Rand Score:  0.001


Cosine Affinity for Agglomerative Clustering

In [15]:
# Cosine distance
cos_agglo = AgglomerativeClustering(n_clusters=3, linkage='average', metric='cosine')
cos_agglo.fit(data)
cos_labels = cos_agglo.labels_

# Jaccard score for each pair of clusters
cos_jaccards = []
for i in range(3):
    for j in range(3):
        cos_jaccards.append(jaccard_score(cos_labels == i, k_labels == j).round(3))

# Make table of jaccard scores
t = pt.PrettyTable()
t.field_names = ['', 'Cluster 1', 'Cluster 2', 'Cluster 3']
t.title = 'Jaccard Score for Each Pair of Clusters (Agglomerative + Cosine)'

for i in range(3):
    t.add_row([f'Cluster {i+1}', cos_jaccards[i], cos_jaccards[i+3], cos_jaccards[i+6]])

print(t)

# Rand score
cos_rand = rand_score(cos_labels, k_labels).round(3)
print("Rand Score: ", cos_rand)

# Adjusted rand score
cos_adj_rand = adjusted_rand_score(cos_labels, k_labels).__round__(3)
print("Adjusted Rand Score: ", cos_adj_rand)

+------------------------------------------------------------------+
| Jaccard Score for Each Pair of Clusters (Agglomerative + Cosine) |
+---------------+---------------+---------------+------------------+
|               |   Cluster 1   |   Cluster 2   |    Cluster 3     |
+---------------+---------------+---------------+------------------+
|   Cluster 1   |     0.342     |      0.0      |      0.018       |
|   Cluster 2   |     0.482     |     0.002     |       0.0        |
|   Cluster 3   |     0.174     |      0.0      |       0.0        |
+---------------+---------------+---------------+------------------+
Rand Score:  0.384
Adjusted Rand Score:  0.001


Similarity between the two Agglomerative distances

In [16]:
# Jaccard score for each pair of clusters
average_jaccards = []
for i in range(3):
    for j in range(3):
        average_jaccards.append(jaccard_score(euc_labels == i, cos_labels == j).round(3))

# Make table of jaccard scores
t = pt.PrettyTable()
t.field_names = ['', 'Cluster 1', 'Cluster 2', 'Cluster 3']
t.title = 'Jaccard Score for Each Pair of Clusters'

for i in range(3):
    t.add_row([f'Cluster {i+1}', average_jaccards[i], average_jaccards[i+3], average_jaccards[i+6]])

print(t)

# Rand score
cos_rand = rand_score(euc_labels, cos_labels).__round__(3)
print("Rand Score: ", cos_rand)

# Adjusted rand score
cos_adj_rand = adjusted_rand_score(euc_labels, cos_labels).__round__(3)
print("Adjusted Rand Score: ", cos_adj_rand)

+-----------------------------------------------+
|    Jaccard Score for Each Pair of Clusters    |
+-----------+-----------+-----------+-----------+
|           | Cluster 1 | Cluster 2 | Cluster 3 |
+-----------+-----------+-----------+-----------+
| Cluster 1 |    1.0    |    0.0    |    0.0    |
| Cluster 2 |    0.0    |    1.0    |    0.0    |
| Cluster 3 |    0.0    |    0.0    |    1.0    |
+-----------+-----------+-----------+-----------+
Rand Score:  1.0
Adjusted Rand Score:  1.0
